![An interactive LADAL notebook](https://slcladal.github.io/images/uq1.jpg)

# Analyzing Co-Occurrences and Collocations

This tutorial is the interactive Jupyter notebook accompanying the [*Language Technology and Data Analysis Laboratory* (LADAL) tutorial *Analyzing Co-Occurrences, N-grams, and Collocations in R*](https://ladal.edu.au/coll.html). 


**Preparation and session set up**

We set up our session by activating the packages we need for this tutorial. 


In [ ]:
# set options
options(warn=-1)  # do not show warnings or messages
# load packages
library(GGally)        # extended functionality in ggplot2
library(ggdendro)      # visualize dendrograms with ggplot2
library(Matrix)        # sparse matrix operations
library(quanteda)      # quantitative analysis of textual data
library(quanteda.textplots)  # additional text visualization tools in quanteda
library(dplyr)         # data manipulation and transformation
library(stringr)       # string manipulation functions
library(tm)            # text mining and analysis
library(tidytext)      # tidy data principles applied to text data
library(readxl)        # reading Excel files
library(writexl)       # saving data in Excel format
library(openxlsx)      # reading and saving Excel files
library(here)          # for generating relative paths


## Using your own data

<div class="warning" style='padding:0.1em; background-color: rgba(215,209,204,.3); color:#51247a'>
<span>
<p style='margin-top:1em; text-align:center'>

While the tutorial uses example data, you can also **use your own data**. To use your own data, click on the folder called `MyTexts` (it is in the menu to the left of the screen) and then simply drag and drop your txt-files into the folder. When you then execute the code chunk below, you will upload your own data and you can then use it in this notebook.<br>
<br>
You can upload <b>only txt-files</b> (simple unformatted files created in or saved by a text editor)! The notebook assumes that you upload some form of text data - not tabular data! <br>
<br>
<b>IMPORTANT</b>: Be sure to <b>replace `mytext` with `text` in the code chunk below and  do not execute the code chunk which loads an example text</b> so that you work with your and not the sample data!</b><br>
</p>
<p style='margin-left:1em;'>
</p></span>
</div>

<br>


In [ ]:
myfiles <- list.files(here::here("notebooks/MyTexts"), # path to the corpus data
                          # full paths - not just the names of the files
                          full.names = T) 
# loop over the vector 'myfiles' that contains paths to the data
mytext <- sapply(myfiles, function(x){

  # read the content of each file using 'scan'
  x <- scan(x, 
            what = "char",    # specify that the input is characters
            sep = "",         # set separator to an empty string (read entire content)
            quote = "",       # set quote to an empty string (no quoting)
            quiet = T,        # suppress scan messages
            skipNul = T)      # skip NUL bytes if encountered

  # combine the character vector into a single string with spaces
  x <- paste0(x, sep = " ", collapse = " ")

  # remove extra whitespaces using 'str_squish' from the 'stringr' package
  x <- stringr::str_squish(x)

})

# inspect the structure of the text object
str(mytext)


## Loading the example data

We begin by loading the data which represents the text of Lewis Caroll's  *Alice's Adventures in Wonderland*. 


In [ ]:
# load text
text <- base::readRDS(url("https://slcladal.github.io/data/alice.rda", "rb"))
# inspect data
head(text)


## Extracting N-Grams

The `quanteda.textstats` package offers the `textstat_collocations` function for extracting N-grams. This function uses the following main arguments

+ `x`: a character, corpus, or tokens object.  
+ `method`: association measure for detecting collocations. Currently this is limited to "lambda".  
+ `size`: integer; the length of the ngram. The default is 2 - if you want to extract tri-grams set `size = 3` and if you want to extract four-grams set `size = 4` and so on.  
+ `min_count`: numeric; minimum frequency of collocations that will be scored.  
+ `smoothing`: numeric; a smoothing parameter added to the observed counts (default is 0.5).  
+ `tolower`: logical; if TRUE, tokens are transformed to lower-case.  


In [ ]:
# concatenate the elements in the 'text' object
text %>% 
  paste0(collapse = " ") %>%
  # convert the concatenated text into tokens
  quanteda::tokens() %>%
  # identify and extract bigrams 
  quanteda.textstats::textstat_collocations(size = 2, min_count = 1) %>%
  # convert into a data frame and save results in an object called 'ngrams'
  as.data.frame() %>%
  # order by frequency
  dplyr::arrange(-count) -> ngrams
# inspect the first 10 rows
head(ngrams, 10)


### Exporting tables

To export a table as an MS Excel spreadsheet, we use `write_xlsx`. 


In [ ]:
# save data for MyOutput folder
write_xlsx(ngrams, here::here("notebooks/MyOutput/ngrams.xlsx"))


<div class="warning" style='padding:0.1em; background-color: rgba(215,209,204,.3); color:#51247a'>
<span>
<p style='margin-top:1em; text-align:center'>
<b>You will find the generated MS Excel spreadsheet named *ngrams.xlsx* in the `MyOutput` folder (located on the left side of the screen).</b> <br><br>Simply double-click the `MyOutput` folder icon, then right-click on the *ngrams.xlsx* file, and choose Download from the dropdown menu to download the file. <br>
</p>
<p style='margin-left:1em;'>
</p></span>
</div>

<br>


## Identifying Collocations

There are various ways of finding collocations depending on the data provided, the context, and the association measure (which represents information of how strong the association between the words is). Below, you will see how to detect collocations in two different data structures:  
+ a list of sentences  
+ concordances  

### Identifying collocations in sentences

In the following, we will find collocations based on their co-occurrence in linguistic units (in this case sentences but this could also be speech units, tweets, or other paragraphs, etc.).

**Data preparation**

In a first step, we split our example text into sentences and clean the data (removing punctuation, converting to lower case, etc.).


In [ ]:
text %>% 
  # concatenate the elements in the 'text' object
  paste0(collapse = " ") %>%
  # separate possessives and contractions
  stringr::str_replace_all(fixed("'"), fixed(" '")) %>%
  stringr::str_replace_all(fixed("’"), fixed(" '")) %>%
  # split text into sentences
  tokenizers::tokenize_sentences() %>%
  # unlist sentences
  unlist() %>%
  # remove non-word characters
  stringr::str_replace_all("\\W", " ") %>%
  stringr::str_replace_all("[^[:alnum:] ]", " ") %>%
  # remove superfluous white spaces
  stringr::str_squish() %>%
  # convert to lower case and save in 'sentences' object
  tolower() -> sentences
# inspect first 10 sentences
head(sentences, 10)


*** 

**Loading your own data**

<div class="warning" style='padding:0.1em; background-color: rgba(215,209,204,.3); color:#51247a'>
<span>
<p style='margin-top:1em; text-align:center'>

While the tutorial uses example data, you can also **use your own data**. To use your own data, click on the folder called `MyTexts` (it is in the menu to the left of the screen) and then simply drag and drop your txt-files into the folder. When you then execute the code chunk below, you will upload your own data and you can then use it in this notebook.<br>
<br>
You can upload <b>only txt-files</b> (simple unformatted files created in or saved by a text editor)! The notebook assumes that you upload some form of text data - not tabular data! <br>
<br>
<b>IMPORTANT</b>: Be sure to <b>replace `mytext` with `sentences` in the code chunk below and  do not execute the code chunk which loads an example text</b> so that you work with your and not the sample data!</b><br>
</p>
<p style='margin-left:1em;'>
</p></span>
</div>

<br>


In [ ]:
myfiles <- list.files(here::here("notebooks/MyTexts"), # path to the corpus data
                          # full paths - not just the names of the files
                          full.names = T) 
# loop over the vector 'myfiles' that contains paths to the data
mytext <- sapply(myfiles, function(x){

  # read the content of each file using 'scan'
  x <- scan(x, 
            what = "char",    # specify that the input is characters
            sep = "",         # set separator to an empty string (read entire content)
            quote = "",       # set quote to an empty string (no quoting)
            quiet = T,        # suppress scan messages
            skipNul = T)      # skip NUL bytes if encountered

  # combine the character vector into a single string with spaces
  x <- paste0(x, sep = " ", collapse = " ")

  # WARNING: OPTIONAL
  # remove non-word characters
  x <- stringr::str_remove_all(x, "[^[:alnum:] ]")
  
  # remove extra white spaces using 'str_squish' from the 'stringr' package
  x <- stringr::str_squish(x)

})

# inspect the structure of the text object
str(mytext)


***

Next, we feed the sentences to a function that tabulates all words and their co-occurrences.


In [ ]:
# load function that generates a co-occurrence table
source("https://slcladal.github.io/rscripts/colldf.R")
cooctb <- colldf(x = sentences)
# inspect results
head(cooctb, 10)


To determine which terms collocate significantly and with what association strength, we use the following information (that is provided by the table above):

* O~11~ = Number of times *word~1~* occurs *with* *word~2~* (\[ w_1 \cap w_2 \])

* O~12~ = Number of times *word~1~* occurs *without* *word~2~* (\[ w_1 \cup w_2 \])

* O~21~ = Number of times `CoocTerm` occurs without `Term`

* O~22~ = Number of terms that are not `coocTerm` or `Term`

Example:


|              | w~2~ present       |     w~2~ absent |      |
 :---          | :-----:    |   --------:  | ---
| **w~1~ present**     | O~11~      | O~12~        |  = R~1~
| **w~1~ absent** | O~21~      | O~22~        |  = R~2~
|              |  = C~1~    |   = C~2~     |  = N |



We could calculate all collocations in the corpus (based on co-occurrence within the same sentence) or we can find collocations of a specific term - here, we will find collocations fo the term *alice*.


In [ ]:
# load function that extracts association statistics
source("https://slcladal.github.io/rscripts/assocstats.R")
# extract association statistics
coocStatz <- assocstats(cooctb, "alice", 10, 5)
# inspect results
coocStatz %>%
  dplyr::mutate_at(9:16, round, 3) %>%
  head(10)


The output shows that the words most strongly associated with *alice* (aside from *alice*) are *said*, *thought*, and *afraid*.


To export a table as an MS Excel spreadsheet, we use `write_xlsx`. 


In [ ]:
# save data for MyOutput folder
write_xlsx(coocStatz, here::here("notebooks/MyOutput/coocStatz.xlsx"))


<div class="warning" style='padding:0.1em; background-color: rgba(215,209,204,.3); color:#51247a'>
<span>
<p style='margin-top:1em; text-align:center'>
<b>You will find the generated MS Excel spreadsheet named *coocStatz.xlsx* in the `MyOutput` folder (located on the left side of the screen).</b> <br><br>Simply double-click the `MyOutput` folder icon, then right-click on the *coocStatz.xlsx* file, and choose Download from the dropdown menu to download the file. <br>
</p>
<p style='margin-left:1em;'>
</p></span>
</div>

<br>

### Identifying collocations using kwics

In this section, we will extract collocations and calculate association measures based on  concordances and the corpus the concordances were extracted from.

We start by cleaning our corpus and splitting it into chapters. 


In [ ]:
# clean corpus
text %>%
  # concatenate the elements in the 'text' object
  paste0(collapse = " ") %>%
  # separate possessives and contractions
  stringr::str_replace_all(fixed("'"), fixed(" '")) %>%
  stringr::str_replace_all(fixed("’"), fixed(" '")) %>%
  # split text into different chapters
  stringr::str_split("CHAPTER [IVX]{1,4}") %>%
  # unlist sentences
  unlist() %>%
  # remove non-word characters
  stringr::str_replace_all("\\W", " ") %>%
  stringr::str_replace_all("[^[:alnum:] ]", " ") %>%
  # remove superfluous white spaces
  stringr::str_squish() %>%
  # convert to lower case and save in 'sentences' object
  tolower() -> texts
# inspect first 100 words the first 10 chapters
head(substr(texts, 1, 100), 10)


<div class="warning" style='padding:0.1em; background-color: rgba(215,209,204,.3); color:#51247a'>
<span>
<p style='margin-top:1em; text-align:center'>
We split the corpus into chapter to mirror the fact that most text data will come in the form of corpora which consist of different files containing texts.
</p>
<p style='margin-left:1em;'>
</p></span>
</div>

<br>


***

<div class="warning" style='padding:0.1em; background-color: rgba(215,209,204,.3); color:#51247a'>
<span>
<p style='margin-top:1em; text-align:center'>

If you have data consisting of cleaned, lower-case, texts with each text being in a different txt-file, you can use your own data! Simply click on the folder called `MyTexts`  (it is in the menu to the left of the screen) and then simply drag and drop your txt-files into the folder. When you then execute the code chunk below, you will upload your own data and you can then use it in this notebook.<br>
<br>
You can upload <b>only txt-files</b> (simple unformatted files created in or saved by a text editor)! The notebook assumes that you upload some form of text data - not tabular data! <br>
<br>
<b>IMPORTANT</b>: Be sure to <b>replace `mytext` with `text` in the code chunk below and  do not execute the code chunk which loads an example text</b> so that you work with your and not the sample data!</b><br>
</p>
<p style='margin-left:1em;'>
</p></span>
</div>

<br>


In [ ]:
myfiles <- list.files(here::here("notebooks/MyTexts"), # path to the corpus data
                          # full paths - not just the names of the files
                          full.names = T) 
# loop over the vector 'myfiles' that contains paths to the data
mytext <- sapply(myfiles, function(x){

  # read the content of each file using 'scan'
  x <- scan(x, 
            what = "char",    # specify that the input is characters
            sep = "",         # set separator to an empty string (read entire content)
            quote = "",       # set quote to an empty string (no quoting)
            quiet = T,        # suppress scan messages
            skipNul = T)      # skip NUL bytes if encountered

  # combine the character vector into a single string with spaces
  x <- paste0(x, sep = " ", collapse = " ")

  # remove extra whitespaces using 'str_squish' from the 'stringr' package
  x <- stringr::str_squish(x)

})

# inspect the structure of the text object
str(mytext)


***

Next, we generate a frequency list of words that occur around a keyword (we use the keyword *alice* in this example but you can also choose a different word).

for this we use the `tokens_select` function (from the `quanteda` package) which has the following arguments: 

+ `x`: a text or collection of texts. The text needs to be tokenised, i.e. split it into individual words, which is why we use the *text* in the `tokens()` function. 
+ `pattern`: a keyword defined by a search pattern  
+ `window`: the size of the context window (how many word before and after)  
+ `valuetype`: the type of pattern matching  
  + "glob" for "glob"-style wildcard expressions;  
  + "regex" for regular expressions; or  
  + "fixed" for exact matching  
+ `selection`: a character to define if the key word should be retained in the resulting frequency list or if it should be removed. The argument offers two options  
  + "keep"  
  + "remove"  
+ `case_insensitive`: logical; if TRUE, ignore case when matching a pattern or dictionary values


In [ ]:
kwic_words <- quanteda::tokens_select(tokens(texts), 
                                      pattern = "alice", 
                                      window = 5, 
                                      selection = "keep") %>%
  unlist() %>%
  # tabulate results
  table() %>%
  # convert into data frame
  as.data.frame() %>%
  # rename columns
  dplyr::rename(token = 1,
                n = 2) %>%
  # add a coulm with type
  dplyr::mutate(type = "kwic")
# inspect results
head(kwic_words, 10)


Next, we create a frequency table of the entire clean corpus.



In [ ]:
# load and execute function that creates frequency lists from text data
source("https://slcladal.github.io/rscripts/corpuswords.R")
corpus_words <- corpuswords(texts)
# inspect the results by displaying the first 10 rows
head(corpus_words, 10)


Next, we combine the two frequency lists.



In [ ]:
# load and execute function that joins 'corpus_words' and 'kwic_words' data frames on the 'token' column
source("https://slcladal.github.io/rscripts/combinefreq.R")
freq_df <- combinefreq(corpus_words, kwic_words) 
# inspect resulting frequency list
head(freq_df, 10)


To determine which terms collocate significantly and with what association strength, we use the following information (that is provided by the table above):

* O11 = Number of times token occurs in `kwic`

* O12 = Number of times token occurs in `corpus` (without `kwic`)

* O21 = Number of times other tokens occur in `kwic`

* O22 = Number of times  other tokens occur in `corpus`

Example:


|              | kwic       |     corpus |      |
 :---          | :-----:    |   --------:  | ---
| **token **     | O~11~      | O~12~        |  = R~1~
| **other tokens** | O~21~      | O~22~        |  = R~2~
|              |  = C~1~    |   = C~2~     |  = N |


In [ ]:
# load and execute function that extracts association statistics
source("https://slcladal.github.io/rscripts/assockwic.R")
coocStatz <- assockwic(freq_df)
# inspect results
head(coocStatz, 10)


## Visualising collocations

### Dotplots

We can now visualize the association strengths in a dotplot as shown in the code chunk below.


In [ ]:
coocStatz %>%
  dplyr::arrange(-phi) %>%
  head(20) %>%
  ggplot(aes(x = reorder(token, phi, mean), y = phi)) +
  geom_point() +
  coord_flip() +
  theme_bw() +
  labs(x = "Token", y = "Association strength (phi)")


### Exporting images

To export image as an png-file, we use `ggsave`. Be aware that we use the `here` function to save the file in the `MyOutput` folder.

The `ggsave` function has the following main arguments:

+ `filename`: File name to create on disk.    
+ `device`: Device to use. Can either be a device function (e.g. png), or one of "eps", "ps", "tex" (pictex), "pdf", "jpeg", "tiff", "png", "bmp", "svg" or "wmf" (windows only). If NULL (default), the device is guessed based on the filename extension  
+ `path`: Path of the directory to save plot to: path and filename are combined to create the fully qualified file name. Defaults to the working directory.  
+ `width, height`: Plot size in units expressed by the units argument. If not supplied, uses the size of the current graphics device.  
+ `units`: One of the following units in which the width and height arguments are expressed: "in", "cm", "mm" or "px".  
+ `dpi`: Plot resolution. Also accepts a string input: "retina" (320), "print" (300), or "screen" (72). Applies only to raster output types.  
+ `bg`: Background color. If NULL, uses the plot.background fill value from the plot theme.  


In [ ]:
# save network graph for MyOutput folder
ggsave(here::here("notebooks/MyOutput/image_01.png"), bg = "white")


<div class="warning" style='padding:0.1em; background-color: rgba(215,209,204,.3); color:#51247a'>
<span>
<p style='margin-top:1em; text-align:center'>
<b>You will find the image-file named *image_01.png* in the `MyOutput` folder (located on the left side of the screen).</b> <br><br>Simply double-click the `MyOutput` folder icon, then right-click on the *image_01.png* file, and choose Download from the dropdown menu to download the file. <br>
</p>
<p style='margin-left:1em;'>
</p></span>
</div>

<br>

### Barplots

We can now visualize the association strengths in a barplot as shown in the code chunk below.


In [ ]:
coocStatz %>%
  dplyr::arrange(-phi) %>%
  head(20) %>%
  ggplot(aes(x = reorder(token, phi, mean), y = phi, label = phi)) +
  geom_bar(stat = "identity") +
  geom_text(aes(y = phi-0.005, label = round(phi, 3)), color = "white", size=3) + 
  coord_flip() +
  theme_bw() +
  labs(x = "Token", y = "Association strength (phi)")


To export the image to the `MyOutput` folder, run the code chunk below (the image will appear as `image_02.png` in the `MyOutput` folder.



In [ ]:
# save network graph for MyOutput folder
ggsave(here::here("notebooks/MyOutput/image_02.png"), bg = "white")


### Dendrograms

Another method for visualizing collocations are dendrograms (tree-diagrams) which show how similarity  to indicate groupings based on numeric values (e.g., association strength). 

We start by extracting the tokens that we want to show (the top 20 collocates of *alice*).


In [ ]:
top20colls <- coocStatz %>%
  dplyr::arrange(-phi) %>%
  head(20) %>%
  dplyr::pull(token)
top20colls


We then need to generate a feature co-occurrence matrix from a document-feature matrix based on the cleaned, lower case sentences of our text.



In [ ]:
keyword_fcm <- sentences %>%
  quanteda::tokens() %>%
  quanteda::dfm() %>%
  quanteda::dfm_select(pattern = c(top20colls, "alice")) %>%
  quanteda::fcm(tri = FALSE)
# inspect
keyword_fcm[1:6, 1:6]


Then we generate the dendrogram based on a  distance matrix generated from the feature co-occurrence matrix.



In [ ]:
hclust(dist(keyword_fcm),       # use distance matrix as data
       method="ward.D2") %>%    # ward.D as linkage method
  ggdendrogram() +              # generate dendrogram
  ggtitle("20 most strongly collocating terms of 'alice'")  # add title


To export the image to the `MyOutput` folder, run the code chunk below (the image will appear as `image_03.png` in the `MyOutput` folder.



In [ ]:
# save network graph for MyOutput folder
ggsave(here::here("notebooks/MyOutput/image_03.png"), bg = "white")


### Network Graphs

Network graphs are a very useful tool to show relationships (or the absence of relationships) between elements. 

To generate a network graph of the collocates of a keyword, we 


In [ ]:
quanteda.textplots::textplot_network(keyword_fcm,  
                                     edge_alpha = 0.8, 
                                     edge_color = "gray",
                                     vertex_labelsize = log(rowSums(keyword_fcm)),
                                     edge_size = 2)


To export the image to the `MyOutput` folder, run the code chunk below (the image will appear as `image_03.png` in the `MyOutput` folder.



In [ ]:
# save network graph for MyOutput folder
ggsave(here::here("notebooks/MyOutput/image_03.png"), bg = "white")


***

[Back to LADAL](https://ladal.edu.au/coll.html)

***
